# 인근 지하철 역 정보 feature 추가

In [8]:
# !pip install geopy

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/PyBioMed-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

from geopy.distance import geodesic

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# Data Load
- X, Y에 대해서 전처리 된 데이터 사용

In [2]:
df = pd.read_csv('../data/preprocessed/1_X_Y_preprocessed.csv')

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_53322/1861683483.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/preprocessed/X_Y_preprocessed.csv')


In [3]:
df.head()

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,시군구 번지
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500.0,0,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500.0,0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,130000.0,0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,117000.0,0,서울특별시 강남구 개포동 658-1


In [106]:
df.tail()

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,시군구 번지,"좌표X,좌표Y"
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,2015-09-09 15:30:27.0,Y,N,127.10672,37.61887,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699"
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,2015-09-09 15:30:27.0,Y,N,127.10672,37.61887,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699"
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,2015-09-09 15:30:27.0,Y,N,127.10672,37.61887,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699"
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,2015-09-09 15:30:27.0,Y,N,127.10672,37.61887,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699"
1128093,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202309,4,13,2014,...,2015-09-09 15:30:27.0,Y,N,127.10672,37.61887,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699"


In [4]:
df['좌표X'].isna().sum()

0

In [67]:
subway = pd.read_csv('../data/subway_feature.csv')
bus = pd.read_csv('../data/bus_feature.csv')

# geopy를 사용하여 역과 아파트 사이의 거리 계산
- '좌표X', '좌표Y'를 합쳐서 '좌표X,좌표Y'변수를 생성한다.
    - 모든 데이터를 다 구하면 너무 오래 걸림
    - unique한 값을 구할 수 있다.
    - 연산 횟수를 줄일 수 있다.

- 각각의 좌표에 대해서 역과의 거리를 구한다.
- 다양한 feature를 생성한다.

In [111]:
# '좌표X', '좌표Y'를 합쳐서 하나의 변수 '좌표X,좌표Y'생성

df['좌표X,좌표Y'] = df['좌표X'].astype(str)+','+df['좌표Y'].astype(str)
print(len(df['좌표X,좌표Y'].unique()))

loc_df = pd.DataFrame(df['좌표X,좌표Y'].unique(), columns=['좌표X,좌표Y']) # '좌표X,좌표Y' 부분만 추출
loc_df

8921


,"좌표X,좌표Y"
0,"127.05721,37.476763"
1,"127.0559898,37.4838941"
2,"127.0766243,37.4962963"
3,"127.0585214,37.4800022"
4,"127.0680279,37.4878016"
...,...
8916,"126.960783522562,37.5003136768081"
8917,"126.935945913625,37.5448750548629"
8918,"127.063427393682,37.5487159907398"
8919,"126.9966,37.5645412"


In [112]:
# 좌표 X, Y 다시 분리

loc_df[['좌표X', '좌표Y']] = loc_df['좌표X,좌표Y'].apply(lambda x: pd.Series(x.split(',')))

loc_df['좌표X'] = loc_df['좌표X'].astype(float)
loc_df['좌표Y'] = loc_df['좌표Y'].astype(float)

loc_df

,"좌표X,좌표Y",좌표X,좌표Y
0,"127.05721,37.476763",127.057210,37.476763
1,"127.0559898,37.4838941",127.055990,37.483894
2,"127.0766243,37.4962963",127.076624,37.496296
3,"127.0585214,37.4800022",127.058521,37.480002
4,"127.0680279,37.4878016",127.068028,37.487802
...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314
8917,"126.935945913625,37.5448750548629",126.935946,37.544875
8918,"127.063427393682,37.5487159907398",127.063427,37.548716
8919,"126.9966,37.5645412",126.996600,37.564541


#### 예시

In [97]:
# 아파트와 인근 지하철 거리 정보 계산
# 반경 m이하의 정류장 개수
# 가장 가까운 정류장

m = 1500

apart_loc = (loc_df.iloc[0]['좌표Y'], loc_df.iloc[0]['좌표X']) # 아파트 위치 정보
re = subway.apply(lambda loc: geodesic(apart_loc, (loc['위도'], loc['경도'])).meters, axis=1) # 아파트와 모든 지하철 정류장 사이의 거리를 구한다.

# 의미있는 정보 표시
print(f'1.5km 이하에 위치한 역 개수: {(re < m).sum()}')
shortest = subway.iloc[re.idxmin()]
print(f'가장 가까운 역과의 거리: {re.min()/1000:.2f}km')
print('가장 가까운 역 이름:', shortest['역사명'])
print('가장 가까운 지하철 라인:', shortest['호선'])

1.5km 이하에 위치한 역 개수: 2
가장 가까운 역과의 거리: 1.13km
가장 가까운 역 이름: 구룡
가장 가까운 지하철 라인: 분당선


## 지하철역

In [113]:
def calculate_dist(loc_df, subway_df):
    apart_loc = (loc_df['좌표Y'], loc_df['좌표X'])
    distances = subway_df.apply(lambda loc: geodesic(apart_loc, (loc['위도'], loc['경도'])).meters, axis=1)
    return distances

distances = loc_df.apply(calculate_dist, subway_df = subway, axis=1)

In [114]:
loc_df['가장 가까운 거리'] = distances.min(axis=1)

In [115]:
loc_df

,"좌표X,좌표Y",좌표X,좌표Y,가장 가까운 거리
0,"127.05721,37.476763",127.057210,37.476763,1127.738351
1,"127.0559898,37.4838941",127.055990,37.483894,413.626689
2,"127.0766243,37.4962963",127.076624,37.496296,401.837945
3,"127.0585214,37.4800022",127.058521,37.480002,759.370910
4,"127.0680279,37.4878016",127.068028,37.487802,221.716698
...,...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314,777.877569
8917,"126.935945913625,37.5448750548629",126.935946,37.544875,451.754539
8918,"127.063427393682,37.5487159907398",127.063427,37.548716,803.720928
8919,"126.9966,37.5645412",126.996600,37.564541,252.032981


In [116]:
min_idx = distances.idxmin(axis=1)
subway.iloc[min_idx].reset_index()

,index,역사_ID,역사명,호선,위도,경도
0,619,1026,구룡,분당선,37.486839,127.058856
1,619,1026,구룡,분당선,37.486839,127.058856
2,679,337,대청,3호선,37.493514,127.079532
3,619,1026,구룡,분당선,37.486839,127.058856
4,618,1027,개포동,분당선,37.489116,127.066140
...,...,...,...,...,...,...
8916,253,2740,숭실대입구(살피재),7호선,37.496029,126.953822
8917,307,2625,광흥창(서강),6호선,37.547456,126.931993
8918,747,211,성수,2호선,37.544581,127.055961
8919,754,204,을지로4가,2호선,37.566595,126.997817


In [117]:
loc_df = pd.concat([loc_df, subway.iloc[min_idx].reset_index()], axis=1)
loc_df

,"좌표X,좌표Y",좌표X,좌표Y,가장 가까운 거리,index,역사_ID,역사명,호선,위도,경도
0,"127.05721,37.476763",127.057210,37.476763,1127.738351,619,1026,구룡,분당선,37.486839,127.058856
1,"127.0559898,37.4838941",127.055990,37.483894,413.626689,619,1026,구룡,분당선,37.486839,127.058856
2,"127.0766243,37.4962963",127.076624,37.496296,401.837945,679,337,대청,3호선,37.493514,127.079532
3,"127.0585214,37.4800022",127.058521,37.480002,759.370910,619,1026,구룡,분당선,37.486839,127.058856
4,"127.0680279,37.4878016",127.068028,37.487802,221.716698,618,1027,개포동,분당선,37.489116,127.066140
...,...,...,...,...,...,...,...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314,777.877569,253,2740,숭실대입구(살피재),7호선,37.496029,126.953822
8917,"126.935945913625,37.5448750548629",126.935946,37.544875,451.754539,307,2625,광흥창(서강),6호선,37.547456,126.931993
8918,"127.063427393682,37.5487159907398",127.063427,37.548716,803.720928,747,211,성수,2호선,37.544581,127.055961
8919,"126.9966,37.5645412",126.996600,37.564541,252.032981,754,204,을지로4가,2호선,37.566595,126.997817


In [118]:
(distances<=1500).sum(axis=1)

0        2
1        7
2        6
3        5
4        8
        ..
8916     3
8917    11
8918     6
8919    19
8920    21
Length: 8921, dtype: int64

In [119]:
loc_df['인근 지하철 역 개수'] = (distances<=1500).sum(axis=1)

In [120]:
loc_df

,"좌표X,좌표Y",좌표X,좌표Y,가장 가까운 거리,index,역사_ID,역사명,호선,위도,경도,인근 지하철 역 개수
0,"127.05721,37.476763",127.057210,37.476763,1127.738351,619,1026,구룡,분당선,37.486839,127.058856,2
1,"127.0559898,37.4838941",127.055990,37.483894,413.626689,619,1026,구룡,분당선,37.486839,127.058856,7
2,"127.0766243,37.4962963",127.076624,37.496296,401.837945,679,337,대청,3호선,37.493514,127.079532,6
3,"127.0585214,37.4800022",127.058521,37.480002,759.370910,619,1026,구룡,분당선,37.486839,127.058856,5
4,"127.0680279,37.4878016",127.068028,37.487802,221.716698,618,1027,개포동,분당선,37.489116,127.066140,8
...,...,...,...,...,...,...,...,...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314,777.877569,253,2740,숭실대입구(살피재),7호선,37.496029,126.953822,3
8917,"126.935945913625,37.5448750548629",126.935946,37.544875,451.754539,307,2625,광흥창(서강),6호선,37.547456,126.931993,11
8918,"127.063427393682,37.5487159907398",127.063427,37.548716,803.720928,747,211,성수,2호선,37.544581,127.055961,6
8919,"126.9966,37.5645412",126.996600,37.564541,252.032981,754,204,을지로4가,2호선,37.566595,126.997817,19


### 원본 데이터에 적용

In [123]:
# '시군구 번지'에 맞춰 X, Y 를 매핑

shortest_dist_mapping = pd.Series({locate: shortest_dist for locate, shortest_dist in zip(loc_df['좌표X,좌표Y'], loc_df['가장 가까운 거리'])})
idx_mapping = pd.Series({locate: idx for locate, idx in zip(loc_df['좌표X,좌표Y'], loc_df['index'])})
ID_mapping = pd.Series({locate: ID for locate, ID in zip(loc_df['좌표X,좌표Y'], loc_df['역사_ID'])})
station_name_mapping = pd.Series({locate: station_name for locate, station_name in zip(loc_df['좌표X,좌표Y'], loc_df['역사명'])})
line_mapping = pd.Series({locate: line for locate, line in zip(loc_df['좌표X,좌표Y'], loc_df['호선'])})
latitude_mapping = pd.Series({locate: latitude for locate, latitude in zip(loc_df['좌표X,좌표Y'], loc_df['위도'])})
longitude_mapping = pd.Series({locate: longitude for locate, longitude in zip(loc_df['좌표X,좌표Y'], loc_df['경도'])})
nums_mapping = pd.Series({locate: nums for locate, nums in zip(loc_df['좌표X,좌표Y'], loc_df['인근 지하철 역 개수'])})

In [121]:
# 새로운 feature 추가
df[['가장 가까운 거리', 'index', '역사_ID', '역사명', '호선', '위도', '경도', '인근 지하철 역 개수']] = np.nan

In [122]:
df

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,시군구 번지,"좌표X,좌표Y",가장 가까운 거리,index,역사_ID,역사명,호선,위도,경도,인근 지하철 역 개수
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# 원본 데이터에 적용

df['가장 가까운 거리'] = df['가장 가까운 거리'].fillna(df['좌표X,좌표Y'].map(shortest_dist_mapping))
df['index'] = df['index'].fillna(df['좌표X,좌표Y'].map(idx_mapping))
df['역사_ID'] = df['역사_ID'].fillna(df['좌표X,좌표Y'].map(ID_mapping))
df['역사명'] = df['역사명'].fillna(df['좌표X,좌표Y'].map(station_name_mapping))
df['호선'] = df['호선'].fillna(df['좌표X,좌표Y'].map(line_mapping))
df['위도'] = df['위도'].fillna(df['좌표X,좌표Y'].map(latitude_mapping))
df['경도'] = df['경도'].fillna(df['좌표X,좌표Y'].map(longitude_mapping))
df['인근 지하철 역 개수'] = df['인근 지하철 역 개수'].fillna(df['좌표X,좌표Y'].map(nums_mapping))

In [126]:
df.iloc[1000000:1000030]

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,시군구 번지,"좌표X,좌표Y",가장 가까운 거리,index,역사_ID,역사명,호선,위도,경도,인근 지하철 역 개수
1000000,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,7,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000001,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,5,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000002,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,6,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000003,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,4,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000004,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,7,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000005,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,11,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000006,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,8,8,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000007,서울특별시 양천구 신정동,1314,1314.0,0.0,중앙하이츠,162.36,201004,21,9,2010,...,서울특별시 양천구 신정동 1314,"126.863710486306,37.5056229099824",774.712928,710.0,248.0,양천구청,2호선,37.512398,126.865819,3.0
1000008,서울특별시 양천구 신정동,958-12,958.0,12.0,지오프레저102동,74.34,200911,2,1,2007,...,서울특별시 양천구 신정동 958-12,"126.858146697438,37.5229775204026",283.062204,368.0,2520.0,신정(은행정),5호선,37.524997,126.856191,5.0
1000009,서울특별시 양천구 신정동,282-16,282.0,16.0,진명,47.95,201001,18,2,1992,...,서울특별시 양천구 신정동 282-16,"126.871796320278,37.5169286580439",729.456330,710.0,248.0,양천구청,2호선,37.512398,126.865819,4.0


In [ ]:
df.to_csv('../data/preprocessed/2_subway_preprocessed.csv')

In [2]:
tmp = pd.read_csv('../data/preprocessed/2_subway_preprocessed.csv')

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_68570/3753150625.py:1: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv('../data/preprocessed/2_subway_preprocessed.csv')


In [3]:
tmp

,Unnamed: 0,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,...,시군구 번지,"좌표X,좌표Y",가장 가까운 거리,index,역사_ID,역사명,호선,위도,경도,인근 지하철 역 개수
0,0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
1,1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
2,2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
3,3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
4,4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
1128090,1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
1128091,1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
1128092,1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
